In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph, END, START
from typing import List
from typing_extensions import TypedDict
from IPython.display import Image, display
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from pprint import pprint


In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [ ]:
llm.invoke("write a balled about LangChain").content

In [ ]:
web_search_tool = TavilySearchResults(k=3)

In [ ]:
urls = [
"https://lilianweng.github.io/posts/2023-06-23-agent/",
"https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
"https://lilianweng.github.io/posts/2023-10-25-adv-attack-1lm/",
]

In [ ]:
docs = [WebBaseLoader(url).load() for url in urls]

docs_list = [item for sublist in docs for item in sublist]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

In [ ]:
doc_splits = text_splitter.split_documents(docs_list)

In [ ]:
# Add to vectordb
vector_store = Chroma.from_documents(
    documents=doc_splits,
    collection_metadata="rag-chroma",
    embedding=embeddings
)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
prompt = hub.pull("rlm/rag-prompt")
print(f"---PROMPT--- {prompt}")

In [ ]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Chain
rag_chain = prompt | llm | StrOutputParser()

In [ ]:
# Run
question = "tell me about agent memory"
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

In [ ]:
class GradeDocuments(BaseModel):
    """ Binary score for relevance check on retrieved documents """

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

In [ ]:
# LLM with function call
structured_11m_grader = llm.with_structured_output(GradeDocuments)
# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(

    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_11m_grader

In [ ]:
question = "tell me about the agent memory."
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

In [ ]:
# Question re_writer
# prompr

system = """You a question re-writer that converts an input question to a better version that is optimized \n
    for web search. Look at the input and try to reason about the underlying semantic intent / meaning. """

re_write_prompt = ChatPromptTemplate.from_messages(

    [
        ("system", system),

        ( 
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        )
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()

In [ ]:
question_rewriter.invoke({"question": question})

In [ ]:
class State(TypedDict):
    """
    Represents the state of our graph.
    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question : str
    generation : str
    web_search: str
    documents: List[str]

In [ ]:
def retrieve(state):
    """
    Retrieve documents
    Args:
        state (dict): The current graph stae

    Returns:
    
    """
    print("---RETRIEVE---")
    question = state["question"]

    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}

In [ ]:
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args :
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
    print("---CHECKING DOCUMENT RELEVANT IS TO QUESTION OR NOR---")

    question = state["question"]
    documents = state["documents"]

    # score each doc
    filtered_docs = []
    web_search = "No"

    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "documents": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)

        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            web_search = "Yes"
            continue

    return {"documents": filtered_docs, "question": question, "web_search": web_search}

In [ ]:
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation

    """

    print(" --- GENERATE --- ")

    question = state["question"]
    documents = state["documents"]

    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

In [ ]:
def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """
    print(" --- TRANSFORM QUERY --- ")
    question = state["question"]
    documents = state["documents"]

    # Re-write question 
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

In [ ]:
def web_search(state):

    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print(" --- WEB SEARCH --- ")

    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})

    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    documents.append(web_results)

    return {"documents": documents, "question": question}

In [ ]:
def decide_to_generate(state):

    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print(" --- ASSESS GRADED DOCUMENTS --- ")
    state["question"]
    web_search = state["web_search"]
    state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
        " --- DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY --- "
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print(" --- DECISION: GENERATE --- ")
        return "generate"

In [ ]:
workflow = StateGraph(State)

# Difine the nodes
workflow.add_node("retrieve", retrieve) 
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)
workflow.add_node("transform_query", transform_query)
workflow.add_node("web_search_node", web_search)

In [ ]:
# Build Graph

workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges("grade_doceuments", decide_to_generate, {"transform_query": "transform_query", "generate": "generate"})
workflow.add_edge("transform_query", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)

app = workflow.compile()

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
inputs = {"question": "tell me about agent memory"}

In [ ]:
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}': ")

        # Optional: print full state at each node
        # pprint(value["keys"], indent=2, width=80, depth=None)

    pprint("\n---\n")
#final generation
pprint(value["generation"])

In [ ]:
inputs_2 = {"question": "tell me about USA"}

In [ ]:
for output in app.stream(inputs_2):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}': ")

        # Optional: print full state at each node
        # pprint(value["keys"], indent=2, width=80, depth=None)

    pprint("\n---\n")
#final generation
pprint(value["generation"])